In [ ]:
#install libraries
!pip install  git+https://github.com/huggingface/transformers.git datasets accelerate scikit-learn -Uqq
!pip install -U transformers>=4.48.0

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,AutoModelForMaskedLM,AutoConfig
from datasets import load_dataset

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.

In [1]:
!python --version

Python 3.11.11


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
)
import string
from sklearn.metrics import f1_score
import gc

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Load and preprocess the dataset
df = pd.read_csv('/content/drive/MyDrive/final_data_1.csv')

# Remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['text'] = df['text'].apply(remove_punctuation)

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.3, random_state=23)

# Convert to HuggingFace Dataset
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)
train_data = train_data.remove_columns(["__index_level_0__"])
test_data = test_data.remove_columns(["__index_level_0__"])

dataset_dict = DatasetDict({
    'train': train_data,
    'test': test_data
})

# Tokenizer and model setup
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

# Reduce max_length to save memory
def tokenizer_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,  # Reduced from 650
        return_tensors="pt"
    )

# Tokenize the dataset
tokenized_train_dataset = dataset_dict["train"].map(tokenizer_function, batched=True)
tokenized_test_dataset = dataset_dict["test"].map(tokenizer_function, batched=True)

# Print a sample
print(tokenized_test_dataset[0])

# Load model config and model for classification
config = AutoConfig.from_pretrained("answerdotai/ModernBERT-base", num_labels=2)
model = AutoModelForSequenceClassification.from_config(config)

# Prepare datasets
train_dataset = tokenized_train_dataset.remove_columns(['text']).rename_column('label', 'labels')
test_dataset = tokenized_test_dataset.remove_columns(['text']).rename_column('label', 'labels')

# Metric function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(labels, predictions, average="weighted")
    return {"f1": score}

# Training arguments with memory-optimized settings
training_args = TrainingArguments(
    output_dir="fine_tuned_modern_bert",
    learning_rate=5e-5,
    per_device_train_batch_size=8,  # Reduced from 32
    per_device_eval_batch_size=8,   # Reduced from 32
    num_train_epochs=3,
    lr_scheduler_type="linear",
    optim="adamw_torch",
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    weight_decay=8e-6,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,  # Mixed precision for memory efficiency
    gradient_checkpointing=True,  # Save GPU memory
    push_to_hub=False,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mabidoye4125 (mabidoye4125-app) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W0407 15:43:10.968000 501 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Epoch,Training Loss,Validation Loss,F1
1,0.284100,0.120337,0.977989
2,0.065200,0.075458,0.986002
3,0.018300,0.104290,0.984669


TrainOutput(global_step=2625, training_loss=0.12251575142996651, metrics={'train_runtime': 1521.1355, 'train_samples_per_second': 13.805, 'train_steps_per_second': 1.726, 'total_flos': 7155914766336000.0, 'train_loss': 0.12251575142996651, 'epoch': 3.0})

In [ ]:
evaluation_results = trainer.evaluate()

print("Evaluation Results:", evaluation_results)

Evaluation Results: {'eval_loss': 0.07545781135559082, 'eval_f1': 0.9860024099804684, 'eval_runtime': 46.2129, 'eval_samples_per_second': 64.917, 'eval_steps_per_second': 8.115, 'epoch': 3.0}


In [ ]:
# Save the trained model
model.save_pretrained("./saved_model_ai")
# Save the tokenizer
tokenizer.save_pretrained("./saved_model_ai")

('./saved_model_ai/tokenizer_config.json',
 './saved_model_ai/special_tokens_map.json',
 './saved_model_ai/tokenizer.json')

In [ ]:
def predict(text):
    # Preprocess input text
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    # Move inputs to GPU if available
    if torch.cuda.is_available():
        inputs = {key: val.cuda() for key, val in inputs.items()}
        model.cuda()

    # Set model to evaluation mode
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze()
        predicted_class_id = probs.argmax().item()

    return (probs[0].item(), probs[1].item(), predicted_class_id)

In [ ]:
some_text = input('Paste: ')

Paste: There’s something magical about Recurrent Neural Networks (RNNs). I still remember when I trained my first recurrent network for Image Captioning. Within a few dozen minutes of training my first baby model (with rather arbitrarily-chosen hyperparameters) started to generate very nice looking descriptions of images that were on the edge of making sense. Sometimes the ratio of how simple your model is to the quality of the results you get out of it blows past your expectations, and this was one of those times. What made this result so shocking at the time was that the common wisdom was that RNNs were supposed to be difficult to train (with more experience I’ve in fact reached the opposite conclusion). Fast forward about a year: I’m training RNNs all the time and I’ve witnessed their power and robustness many times, and yet their magical outputs still find ways of amusing me. This post is about sharing some of that magic with you.  We’ll train RNNs to generate text character by cha

In [ ]:
predict(some_text)

(0.9987527132034302, 0.0012472739908844233, 0)